In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
# Loading data
data_no_onehot = pd.read_csv('../data/all.csv')
data_onehot = pd.read_csv('../data/all_with_onehot.csv')

In [3]:
data_no_onehot.head()

,Date,Weekday,MealType,Paytm+Cash,Coupons,SemType,Holiday,CouponsMand
0,2022-09-01,3,BreakFast,186.0,117.0,Acad,0,0.0
1,2022-09-01,3,Lunch,293.0,217.0,Acad,0,0.0
2,2022-09-01,3,EveningSnacks,37.0,139.0,Acad,0,0.0
3,2022-09-01,3,Dinner,113.0,220.0,Acad,0,0.0
4,2022-09-02,4,BreakFast,100.0,236.0,Acad,0,0.0


In [4]:
# Preprocessing the dataset
def preprocess(data, to_onehot=True, to_drop_vacation=False):
    if not to_onehot:
        data = data.dropna()
        
    # Converting the date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Adding day, dayofweek, month and year columns
    data['Day'] = pd.DatetimeIndex(data['Date']).day
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    data['Year'] = pd.DatetimeIndex(data['Date']).year

    # Scaling the data
    to_normalize = ['Day']
    scaler = MinMaxScaler()
    data[to_normalize] = scaler.fit_transform(data[to_normalize])

    to_normalize = ['Holiday']
    scaler = StandardScaler()
    data[to_normalize] = scaler.fit_transform(data[to_normalize])

    # Encoding the categorical data
    if to_onehot:
        categorical_features = ['Weekday', 'Month', 'Year', 'MealType', 'SemType']
        data = pd.get_dummies(data, columns=categorical_features)
    else:
        # Using label encoding
        categorical_features = ['MealType', 'SemType']
        for feature in categorical_features:
            data[feature] = data[feature].astype('category')
            data[feature] = data[feature].cat.codes

    # Dropping vacations (if required)
    if to_drop_vacation:
        data = data[data['Semtype_Vacation'] == 0]

    # Splitting into X and y
    X = data.drop(columns=['Paytm+Cash', 'Coupons'])
    y_paytm = data['Paytm+Cash']
    y_coupons = data['Coupons']
    y_total = data['Paytm+Cash'] + data['Coupons']

    return X, y_paytm, y_coupons, y_total

In [5]:
# Splitting the data into train and test sets (with vacations)
X_no_onehot, y_paytm_no_onehot, y_coupons_no_onehot, y_total_no_onehot = preprocess(data_no_onehot, to_onehot=False)
X_onehot, y_paytm_onehot, y_coupons_onehot, y_total_onehot = preprocess(data_onehot)

# cutoff at 2023-08-31 for no onehot
X_train_no_onehot = X_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
X_test_no_onehot = X_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_paytm_train_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_paytm_test_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_coupons_train_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_coupons_test_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_total_train_no_onehot = y_total_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_total_test_no_onehot = y_total_no_onehot[X_no_onehot['Date'] > '2023-08-31']

# cutoff at 2023-08-31 for onehot
X_train_onehot = X_onehot[X_onehot['Date'] <= '2023-08-31']
X_test_onehot = X_onehot[X_onehot['Date'] > '2023-08-31']
y_paytm_train_onehot = y_paytm_onehot[X_onehot['Date'] <= '2023-08-31']
y_paytm_test_onehot = y_paytm_onehot[X_onehot['Date'] > '2023-08-31']
y_coupons_train_onehot = y_coupons_onehot[X_onehot['Date'] <= '2023-08-31']
y_coupons_test_onehot = y_coupons_onehot[X_onehot['Date'] > '2023-08-31']
y_total_train_onehot = y_total_onehot[X_onehot['Date'] <= '2023-08-31']
y_total_test_onehot = y_total_onehot[X_onehot['Date'] > '2023-08-31']

# Drop the date column
X_train_no_onehot = X_train_no_onehot.drop(columns=['Date'])
X_test_no_onehot = X_test_no_onehot.drop(columns=['Date'])
X_train_onehot = X_train_onehot.drop(columns=['Date'])
X_test_onehot = X_test_onehot.drop(columns=['Date'])

X_train_no_onehot.head(10)

/var/folders/4d/w8t5vd_11zs7s9jlct5ksrfr0000gn/T/ipykernel_22711/2238380120.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])
/var/folders/4d/w8t5vd_11zs7s9jlct5ksrfr0000gn/T/ipykernel_22711/2238380120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Day'] = pd.DatetimeIndex(data['Date']).day
/var/folders/4d/w8t5vd_11zs7s9jlct5ksrfr0000gn/T/ipykernel_22711/2238380120.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,Weekday,MealType,SemType,Holiday,CouponsMand,Day,Month,Year
0,3,0,0,-0.395613,0.0,0.000000,9,2022
1,3,3,0,-0.395613,0.0,0.000000,9,2022
2,3,2,0,-0.395613,0.0,0.000000,9,2022
3,3,1,0,-0.395613,0.0,0.000000,9,2022
4,4,0,0,-0.395613,0.0,0.033333,9,2022
5,4,3,0,-0.395613,0.0,0.033333,9,2022
6,4,2,0,-0.395613,0.0,0.033333,9,2022
7,4,1,0,-0.395613,0.0,0.033333,9,2022
8,5,0,0,-0.247439,0.0,0.066667,9,2022
9,5,3,0,-0.247439,0.0,0.066667,9,2022


In [6]:
# Train an MLP Regressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

# MLP for Paytm+Cash
mlp_paytm = MLPRegressor()

# MLP for Coupons
mlp_coupons = MLPRegressor()

# MLP for Total
mlp_total = MLPRegressor()

# Grid search for Paytm+Cash
parameters = {
    'hidden_layer_sizes': [(200, 200), (200, 100), (100, 100), (50, 50), (300, 300), (300, 200), (400, 400)],
    'activation': [
        'relu',
        # 'logistic'
        ],
    'solver': [
        'adam', 
        # 'sgd'
        ],
    'learning_rate': [
        # 'constant',
        'adaptive'
    ],
    'max_iter': [1500],
    'alpha': [0.0001, 0.001, 0.01, 0.1]
}


def grid_search(X_train, y_train, X_test, y_test, parameters):
   # Grid search for paytm+cash using for loops
    best_parameters = {}

    for hidden_layer_sizes in parameters['hidden_layer_sizes']:
        for activation in parameters['activation']:
            for solver in parameters['solver']:
                for learning_rate in parameters['learning_rate']:
                    for max_iter in parameters['max_iter']:
                        for alpha in parameters['alpha']:
                            print("Parameters:", hidden_layer_sizes, activation, solver, learning_rate, max_iter, alpha)
                            mlp = MLPRegressor(
                                hidden_layer_sizes=hidden_layer_sizes, 
                                activation=activation, solver=solver, 
                                learning_rate=learning_rate, 
                                max_iter=max_iter,
                                alpha=alpha
                                )
                            mlp.fit(X_train, y_train)
                            # training error:
                            y_pred = mlp.predict(X_train)
                            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
                            r2_train = r2_score(y_train, y_pred)
                            print('Training error:' + str(rmse_train) + ' ' + str(r2_train))
                            # testing error:
                            y_pred = mlp.predict(X_test)
                            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                            r2 = r2_score(y_test, y_pred)
                            print('Testing error:' + str(rmse) + ' ' + str(r2) + '\n')
                            best_parameters[(rmse, r2)] = (hidden_layer_sizes, activation, solver, learning_rate, max_iter)
        
    # Printing the best parameters with scores
    best_parameters = sorted(best_parameters.items(), key=lambda x: -1 * x[0][1])
    print('Best parameters:')
    print('RMSE:', best_parameters[0][0][0])
    print('R2:', best_parameters[0][0][1])
    print('Parameters:', best_parameters[0][1])

    return best_parameters[0][1]

In [7]:
# Grid search for Paytm+Cash onehot using grid search
best_parameters_paytm_onehot = grid_search(X_train_onehot, y_paytm_train_onehot, X_test_onehot, y_paytm_test_onehot, parameters)


Parameters: (200, 200) relu adam adaptive 1500 0.0001
Training error:3.670643136929812 0.9922725894520033
Testing error:20.318682881244325 0.7141699899094822

Parameters: (200, 200) relu adam adaptive 1500 0.001
Training error:3.667026341726779 0.9922878100502088
Testing error:18.22828029614229 0.7699574800283359

Parameters: (200, 200) relu adam adaptive 1500 0.01
Training error:2.6774037216513373 0.9958887168933943
Testing error:21.004421459498026 0.6945513795125129

Parameters: (200, 200) relu adam adaptive 1500 0.1
Training error:4.173688195321394 0.9900094439594915
Testing error:18.20145763108587 0.7706339907651047

Parameters: (200, 100) relu adam adaptive 1500 0.0001
Training error:4.653517740038537 0.9875802628101163
Testing error:20.00032532332106 0.7230567145854174

Parameters: (200, 100) relu adam adaptive 1500 0.001
Training error:2.979544383004738 0.9949084574644277
Testing error:17.06678257789931 0.7983398751030689

Parameters: (200, 100) relu adam adaptive 1500 0.01
Trai

In [8]:
# Grid search for Coupons onehot using grid search
best_parameters_coupons_onehot = grid_search(X_train_onehot, y_coupons_train_onehot, X_test_onehot, y_coupons_test_onehot, parameters)

Parameters: (200, 200) relu adam adaptive 1500 0.0001
Training error:6.4859146589398815 0.9946949716701632
Testing error:78.08379318453386 0.28304380060761025

Parameters: (200, 200) relu adam adaptive 1500 0.001
Training error:6.00633816653764 0.995450488076458
Testing error:65.94785091501507 0.48858658686174095

Parameters: (200, 200) relu adam adaptive 1500 0.01
Training error:9.434827579762153 0.9887743064223632
Testing error:61.70363098706068 0.5522946732713496

Parameters: (200, 200) relu adam adaptive 1500 0.1
Training error:28.30443245820908 0.898969116751845
Testing error:59.308849298480524 0.5863721092420566

Parameters: (200, 100) relu adam adaptive 1500 0.0001
Training error:8.259432986389887 0.9913970833504573
Testing error:50.69555551827309 0.6977887743824619

Parameters: (200, 100) relu adam adaptive 1500 0.001
Training error:8.912647141534876 0.989982515637762
Testing error:61.08724871469441 0.561194613085017

Parameters: (200, 100) relu adam adaptive 1500 0.01
Training

/opt/homebrew/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  warnings.warn(


Training error:16.874251520764606 0.9640917463758261
Testing error:77.32241564451297 0.2969573920489814

Parameters: (50, 50) relu adam adaptive 1500 0.001
Training error:18.22260192829021 0.9581239197120336
Testing error:64.1728998199313 0.5157449612654338

Parameters: (50, 50) relu adam adaptive 1500 0.01
Training error:26.865023993755194 0.9089835875595299
Testing error:67.2392897590799 0.4683607285857657

Parameters: (50, 50) relu adam adaptive 1500 0.1


/opt/homebrew/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  warnings.warn(


Training error:11.453701746244654 0.9834561346255432
Testing error:51.495367207804094 0.6881777232630568

Parameters: (300, 300) relu adam adaptive 1500 0.0001
Training error:6.218111130325315 0.9951240167334876
Testing error:65.37568158585326 0.49742222558217

Parameters: (300, 300) relu adam adaptive 1500 0.001
Training error:6.187554160684991 0.995171821983058
Testing error:76.06405133881569 0.31965417238032967

Parameters: (300, 300) relu adam adaptive 1500 0.01
Training error:6.834885104845157 0.9941087468582002
Testing error:68.75166273492057 0.4441760821042525

Parameters: (300, 300) relu adam adaptive 1500 0.1
Training error:5.897740546010982 0.9956135157559632
Testing error:62.21734816674024 0.5448088469765138

Parameters: (300, 200) relu adam adaptive 1500 0.0001
Training error:7.399306575760048 0.9930955785805309
Testing error:73.28961881911495 0.36838018629656366

Parameters: (300, 200) relu adam adaptive 1500 0.001
Training error:5.208888041454672 0.9965783522865143
Testin

In [9]:
# Grid search for Paytm+Cash no onehot using grid search
best_parameters_paytm_no_onehot = grid_search(X_train_no_onehot, y_paytm_train_no_onehot, X_test_no_onehot, y_paytm_test_no_onehot, parameters)

Parameters: (200, 200) relu adam adaptive 1500 0.0001
Training error:42.65866424683448 -0.04367231461119325
Testing error:38.954438523047045 -0.05058360250929539

Parameters: (200, 200) relu adam adaptive 1500 0.001
Training error:43.71848590865923 -0.09617497633620053
Testing error:38.77518705305782 -0.04093718573428262

Parameters: (200, 200) relu adam adaptive 1500 0.01
Training error:41.66728182249278 0.004273653902776675
Testing error:37.91236858733494 0.0048728938728261895

Parameters: (200, 200) relu adam adaptive 1500 0.1
Training error:41.685304971230025 0.0034120663672894125
Testing error:38.132115132330874 -0.006696390373986727

Parameters: (200, 100) relu adam adaptive 1500 0.0001
Training error:41.729880902206986 0.0012795361073545397
Testing error:38.162252451679464 -0.008288283171232091

Parameters: (200, 100) relu adam adaptive 1500 0.001
Training error:41.03462712489404 0.03428130002257168
Testing error:37.07532938761807 0.0483291730959704

Parameters: (200, 100) relu 

In [10]:
# Grid search for Coupons no onehot using grid search
best_parameters_coupons_no_onehot = grid_search(X_train_no_onehot, y_coupons_train_no_onehot, X_test_no_onehot, y_coupons_test_no_onehot, parameters)

Parameters: (200, 200) relu adam adaptive 1500 0.0001
Training error:69.90430251965783 0.3837554929399285
Testing error:94.68677518015971 -0.05426422791395935

Parameters: (200, 200) relu adam adaptive 1500 0.001
Training error:76.80374354436351 0.2561077655281686
Testing error:114.92406787337752 -0.5530763246419999

Parameters: (200, 200) relu adam adaptive 1500 0.01
Training error:70.07298207079981 0.38077790053089533
Testing error:95.25679432301044 -0.06699588272769041

Parameters: (200, 200) relu adam adaptive 1500 0.1
Training error:73.3422570958776 0.32165006165613286
Testing error:108.48466233907202 -0.3839089006717915

Parameters: (200, 100) relu adam adaptive 1500 0.0001
Training error:72.33717648327371 0.3401148522820756
Testing error:91.81596765650102 0.008695105076890086

Parameters: (200, 100) relu adam adaptive 1500 0.001
Training error:70.09279402832942 0.38042770175644247
Testing error:97.08743923404874 -0.10840100798676144

Parameters: (200, 100) relu adam adaptive 150

In [11]:
# # Grid search for Paytm+Cash using onehot
# mlp_paytm = MLPRegressor()
# clf_paytm = GridSearchCV(mlp_paytm, parameters, n_jobs=-1, verbose=10, cv=3, return_train_score=True)
# clf_paytm.fit(X_train_onehot, y_paytm_train_onehot)
# print('Best parameters for Paytm+Cash onehot: ', clf_paytm.best_params_)
# print('Results for Paytm+Cash onehot: ', clf_paytm.cv_results_)

# # Metrics
# y_paytm_pred_onehot = clf_paytm.predict(X_test_onehot)
# print('Paytm+Cash onehot Metrics:')
# print('RMSE: ', np.sqrt(mean_squared_error(y_paytm_test_onehot, y_paytm_pred_onehot)))
# print('R2 Score: ', r2_score(y_paytm_test_onehot, y_paytm_pred_onehot))



In [12]:
# # Grid search for Paytm+Cash using no onehot
# mlp_paytm = MLPRegressor()
# clf_paytm = GridSearchCV(mlp_paytm, parameters, n_jobs=-1, verbose=10, cv=3, return_train_score=True)
# clf_paytm.fit(X_train_no_onehot, y_paytm_train_no_onehot)
# print('Best parameters for Paytm+Cash no onehot: ', clf_paytm.best_params_)
# print('Results for Paytm+Cash no onehot: ', clf_paytm.cv_results_)

# # Metrics
# y_paytm_pred_no_onehot = clf_paytm.predict(X_test_no_onehot)
# print('Paytm+Cash no onehot Metrics:')
# print('RMSE: ', np.sqrt(mean_squared_error(y_paytm_test_no_onehot, y_paytm_pred_no_onehot)))
# print('R2 Score: ', r2_score(y_paytm_test_no_onehot, y_paytm_pred_no_onehot))


In [13]:
# # Grid search for Coupons using onehot
# mlp_coupons = MLPRegressor()
# clf_coupons = GridSearchCV(mlp_coupons, parameters, n_jobs=-1, verbose=10, cv=3, return_train_score=True)
# clf_coupons.fit(X_train_onehot, y_coupons_train_onehot)
# print('Best parameters for Coupons onehot: ', clf_coupons.best_params_)
# print('Results for Coupons onehot: ', clf_coupons.cv_results_)

# # Metrics
# y_coupons_pred_onehot = clf_coupons.predict(X_test_onehot)
# print('Coupons onehot Metrics:')
# print('RMSE: ', np.sqrt(mean_squared_error(y_coupons_test_onehot, y_coupons_pred_onehot)))
# print('R2 Score: ', r2_score(y_coupons_test_onehot, y_coupons_pred_onehot))


In [14]:
# # Grid search for Coupons using no onehot
# mlp_coupons = MLPRegressor()
# clf_coupons = GridSearchCV(mlp_coupons, parameters, n_jobs=-1, verbose=10, cv=3, return_train_score=True)
# clf_coupons.fit(X_train_no_onehot, y_coupons_train_no_onehot)
# print('Best parameters for Coupons no onehot: ', clf_coupons.best_params_)
# print('Results for Coupons no onehot: ', clf_coupons.cv_results_)

# # Metrics
# print('Coupons no onehot Metrics:')
# y_coupons_pred_no_onehot = clf_coupons.predict(X_test_no_onehot)
# print('RMSE: ', np.sqrt(mean_squared_error(y_coupons_test_no_onehot, y_coupons_pred_no_onehot)))
# print('R2 Score: ', r2_score(y_coupons_test_no_onehot, y_coupons_pred_no_onehot))
